In [1]:
import numpy as np
import pandas as pd
import pandas.api.types
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import GroupKFold, StratifiedGroupKFold

import lightgbm as lgb

In [2]:
import h5py
from io import BytesIO
from PIL import Image
from matplotlib import pyplot as plt
import wandb
import copy
import csv

import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.transforms import v2

from torchvision.models import resnet50, ResNet50_Weights, efficientnet_b4, EfficientNet_B4_Weights, efficientnet_b0, EfficientNet_B0_Weights


In [3]:
class TestDataset(Dataset):
    def __init__(self, dataframe, hdfPath, augmentation=None):
        self.dataframe = dataframe
        self.hdf = h5py.File(hdfPath, mode="r")
        self.ids = dataframe['isic_id']
        self.augmentation = augmentation
        
    def __len__(self):
        return len(self.ids)
    
    def __getitem__(self, idx):
        itemId = self.ids[idx]
        image = Image.open(BytesIO(self.hdf[itemId][()]))
        image = np.array(image)
            
        if self.augmentation:
            image = self.augmentation(image=image)['image']
            
        return image, itemId
    
class customModel(torch.nn.Module):
    def __init__(self, pretrainedModel):
        super(customModel, self).__init__()
        self.pretrainedModel = pretrainedModel
        
        self.classifier = torch.nn.Sequential(
            torch.nn.Dropout(p=0.25),
            torch.nn.Linear(1000, 1),
            torch.nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.pretrainedModel(x)
        x = self.classifier(x)
        return x

In [4]:
train_metadata = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv')
test_metadata = pd.read_csv('/kaggle/input/isic-2024-challenge/test-metadata.csv')

transforms_test = A.Compose([
    A.Resize(224, 224, interpolation=2),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

train = TestDataset(train_metadata, '/kaggle/input/isic-2024-challenge/train-image.hdf5', augmentation=transforms_test)
test = TestDataset(test_metadata, '/kaggle/input/isic-2024-challenge/test-image.hdf5', augmentation=transforms_test)

trainLoader = DataLoader(train, batch_size=32, shuffle=False)
testLoader = DataLoader(test, batch_size=32, shuffle=False)


/tmp/ipykernel_23/725557698.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_metadata = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv')


In [5]:
if torch.cuda.is_available():
    print("GPU is available")
    device = torch.device("cuda")
else:
    print("GPU is not available, using CPU instead")
    device = torch.device("cpu")


GPU is available


In [6]:
pretrainedModel = efficientnet_b0()
model = customModel(pretrainedModel)
model.load_state_dict(torch.load('/kaggle/input/skin-cancer-model-training-w-additional-data/model_params_epoch_2.pt'))
model = model.to(device)


/tmp/ipykernel_23/2343000053.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/kaggle/input/skin-cancer-model-training-w-additional-data

In [7]:
nnTrainPreds = np.array([])

model.eval()
with torch.no_grad():
    for images, cases in trainLoader:
        images = images.to(device)
        preds = model(images)

        preds = preds.squeeze(dim=1).detach().cpu().numpy()
        
        nnTrainPreds = np.append(nnTrainPreds, preds)

In [8]:
df = pd.DataFrame(nnTrainPreds)
df.to_csv('preds.csv')